In [ ]:
from collections import namedtuple
import sys
sys.path.append('../')
from task_graph import TaskGraph

In [ ]:
task_graph = TaskGraph("Create haikus on all seasons")

In [ ]:
# Example Usage
task_graph.add_task(
    task_name='Fall haiku', 
    agent_name='haiku_agent',
    task_description='Write a haiku about the Fall season',
    )

task_graph.add_task(
    task_name='Winter haiku', 
    agent_name='haiku_agent',
    task_description='Write a haiku about winter',
    )
task_graph.add_task(
    task_name='Get Data', 
    agent_name='haiku_agent',
    task_description='Write a haiku about the Fall season',
    )

task_graph.add_task(
    task_name='Upload data', 
    agent_name='haiku_agent',
    task_description='Write a haiku about winter',
    )

In [ ]:
task_graph.add_dependency('Winter haiku', 'Fall haiku')
task_graph.add_dependency('Get Data', 'Upload data')

In [ ]:
print(task_graph.generate_markdown())

In [ ]:
task_info = task_graph.graph.nodes['Fall haiku']

In [ ]:
task_info['task'].task_description

In [ ]:
from task import Task

In [ ]:
orchestration_task_desc = f"""
You are helping the user complete the task: {task_graph.name}.  It has {task_graph.graph.number_of_nodes()}.
"""

In [ ]:
x = Task(
    task_name='orchestration',
    agent_name='agent_manager',
    task_description=orchestration_task_desc,
    )

In [ ]:
x.get_system_instruction()

In [ ]:
task_graph.add_task('Task1.1', sequence=2)

In [ ]:
task_graph.add_dependency('Task1', 'Task1.1')

In [ ]:
task_graph.update_task('Task1.1', completed=True)

In [ ]:
out = task_graph.get_next_task('Task1.1')
print(out)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
query = 'travis+coan'
max_results = 1

In [ ]:
api_url = f"http://export.arxiv.org/api/query?search_query={query}&start=0&max_results={max_results}"
# Get the response
response = requests.get(api_url)
# Get the content
content = response.content
# Parse the content
soup = BeautifulSoup(content, 'xml')

# Get the entries
entries = soup.find_all('entry')

In [ ]:
results = []
for entry in entries:
    title = entry.title.text
    authors = [author.find('name').text for author in entry.find_all('author')]
    summary = entry.summary.text
    link = entry.link['href']
    date_of_publication = entry.published.text

    result = {
        'title': title,
        'author': authors,
        'summary': summary,
        'link': link,
        'date_of_publication': date_of_publication
    }
    results.append(result)

In [ ]:
results

In [1]:
import requests

In [2]:
res = requests.get('https://arxiv.org/pdf/2311.05661v1.pdf')

In [3]:
# View the pdf content in pdf viewer
with open('test.pdf', 'wb') as f:
    f.write(res.content)

In [ ]:
with ('~/Downloads/test.pdf', 'wb') as f:
    f.write(res.content)

In [14]:
# read the pdf and get the text
from PyPDF2 import PdfReader

reader = PdfReader('test.pdf', 'rb')

number_of_pages = len(reader.pages)

full_text = ''
for page in reader.pages:
    full_text +=   page.extract_text()


In [17]:
import re

def remove_image_chars(text):
    # The image characters are in the format /uniXXXX where XXXX is a hexidecimal number, e.g. /uniE0BB
    # So, we need to define a regular expression pattern to match these sequences
    # Define a regular expression pattern to match Unicode sequences
    pattern = re.compile(r'/uni[0-9A-Fa-f]{6}')

    # Use the sub() function to replace matches with an empty string
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [19]:
print(remove_unicode_chars(full_text))

Preprint
PROMPT ENGINEERING A PROMPT ENGINEER
Qinyuan Ye1†Maxamed Axmed2Reid Pryzant2Fereshte Khani2
1University of Southern California2Microsoft
qinyuany@usc.edu fkhani@microsoft.com
ABSTRACT
Prompt engineering is a challenging yet crucial task for optimizing the perfor-
mance of large language models (LLMs). It requires complex reasoning to exam-
ine the model’s errors, hypothesize what is missing or misleading in the current
prompt, and communicate the task with clarity. While recent works indicate that
LLMs can be meta-prompted to perform automatic prompt engineering, their po-
tentials may not be fully untapped due to the lack of sufficient guidance to elicit
complex reasoning capabilities in LLMs in the meta-prompt. In this work, we in-
vestigate the problem of “prompt engineering a prompt engineer”—constructing
a meta-prompt that more effectively guides LLMs to perform automatic prompt
engineering. We introduce and analyze key components, such as a step-by-step
reasoning templat

In [5]:
pip install pyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.3 MB/s eta 0:00:001.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
